<a href="https://colab.research.google.com/github/edogriba/FOR-2024/blob/main/Homework02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lot sizing

A company has to plan the production of 3 products, A1, A2, A3, for a time horizon of four months, from January to April. The working days are: 22 for January, 20 for February, 23 for March, 22 for April.

Sales forecasts indicate the following maximum demand, per product and month.


Demand|January|February|March|April
------|-------|--------|-----|-----
A1| 5300| 1200| 7400| 5300
A2| 4500| 5400| 6500| 7200
A3| 4400| 6700| 12500| 13200

The following table reports the price of each product (Euro) and its unit production cost (Euro). It also reports the maximum number of pieces that can be produced in a single day (pieces/day), if the whole production capability of the factory is used to produce units of that product.

Product|A1|A2|A3
-------|--|--|--
Price| 124| 109| 115
Production cost| 75| 53| 65
Production amount| 500| 450| 550

Inventory can be used to store units of unsold product. The inventory cost per month and unit
is 3 for product A1, 4 for product A2, and 2 for product A3. Each month, no more than 800
total units of the three products can be stored.

1. Give an integer linear programming formulation for the problem of determining a production plan that maximizes the total revenue.
2. Integrality restrictions are mandatory for this problem, since we are dealing with discrete products. In spite of this, when dealing with problems involving large quantities of product, it is often possible, when dropping the integrality constraints, to obtain solutions that are almost integer. Assess, computationally, the difference between integer and continuous optimal solutions for the original formulation.

**Sets**

$I$ = set of products

$J$ = set of months or time periods

**Parameters**

$b_j$ = working days of J

$d_{ij}$ = demand of product $ i \in I$ of month $ j \in J$

$p_i$ = unit price of product $i \in I$

$c_i$ = unit cost of product $i\in I$

$q_i$ = production limit for product $i \in I$

$f_i$ = inventory cost of product $i \in I$

$K$ = maximum amount of inventory per period

**Decision variables**

$x_{ij}$ = number of product $i \in I$ produced in period $j \in J$

$z_{ij}$ = number of inventory of product $ i \in I$ in period $j \in J$

$v_{ij}$ = number of products $ i \in I $ sold in period $ j \in J$

All these variables are integers

**Objective function**

$$ \max \sum_{j \in J} \sum_{i \in I} p_i v_{ij}
 - c_i x_{ij} - f_i z{ij}$$

 <h3 align="center">Formulation</h3>

- Sets
    - $I$: products
    - $J=$ {1,$\ldots,n$}: months, $n=4$
    
- Parameters
    - $b_j$: number of working days for month $j \in J$
    - $d_{ij}$: maximum demand for product $i$ in month $j,i\in I,j \in J$
    - $r_i$: unit price for product $i \in I$
    - $c_i$: unit production cost for product $i \in I$
    - $q_i$: maximum production level for product $i \in I$
    - $m_i$: unit inventory cost, per month, for product $i \in I $
    - $K$: inventory capacity
    
- Variables
    - $x_{ij}$: quantity of product $i$ produced in month $j$, $i \in I, j \in J$
    - $v_{ij}$: quantity of product $i$ sold in month $j$, $i \in I, j \in J$
    - $z_{ij}$: quantity of product $i$ stored at the end of month $j$, $i \in I, j \in J \cup \{0\}$
    
    
- Model
$$
\begin{array}{lll}
  \min & \sum_{i \in I,j\in J} r_i v_{ij} - c_{i} x_{ij} - m_i z_{ij} \qquad & & \text{(revenue)}\\
  \textrm{s. t.}
  & v_{ij} \leq d_{ij} & i \in I, j \in J & \text{(demand)}\\
  & \sum_{i \in I} \frac{x_{ij}}{q_i} \leq b_j & j \in J & \text{(production)}\\
  & z_{i,j-1} + x_{ij} = z_{ij} + v_{ij} & i \in I, j \in J & \text{(balance)}\\
  & z_{i,0} = 0 & i \in I & \text{(init)}\\
  & \sum_{i \in I} z_{ij} \leq K & j \in J & \text{(capacity)}\\
  & x_{ij} \in \mathbb{Z}^{+} & i \in I, j \in J &  \text{(nonnegativity, integrality)}\\
  & v_{ij} \geq 0 & i\in I, j \in J & \text{(nonnegativity)}\\
  & z_{ij} \geq 0 & i\in I, j\in I\cup\{0\} & \text{(nonnegativity)}
\end{array}
$$

The introduction of variable $z_{ij}$ for month 0 (constraint init) is necessary for the correctness of the balance constraint. Note that it suffices to impose the integrality on $x_{ij}$ to guarantee that $v_{ij}$ and $z_{ij}$ will be integral as well because of the balance constraint.


<h3>Optional questions<h3>

1. Give a formulation for the variant where a minimum lot size is required whenever a product, per month, is produced, and where a fixed cost is charged, per month and product, whenever the production line for the corresponding product is active. Use the data:

Product|A1|A2|A3
-------|--|--|--
Fixed cost|150000|150000|100000
Minimum lot size|20|20|16


2. Assess the effect of integrality for the variant of the problem. Do you expect the difference between the integer and continuous solutions to be larger in this case?

In [213]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [214]:
import mip
from mip import BINARY,INTEGER
# Number of products
n_product = 3
# Number of months
n_months = 4

# Set of products
I = range(n_product)
# Set of months
J = range(1, n_months+1)

# Working days per month
b = [22, 20, 23, 22]
# Maximum demand, per product and month
d = [[5300, 1200, 7400, 5300], [4500, 5400, 6500, 7200], [4400, 6700, 12500, 13200]]

# Price of each product (Euro)
r = [124, 109, 115]
# Unit production cost (Euro)
c = [75, 53, 65]

# Maximum number of pieces that can be produced in a single day (pieces/day)
q = [500, 450, 550]
# Inventory cost per month and unit
m = [3, 4, 2]

# Maximum number of total units of the three products that can be stored
K = 800

# Fixed cost charged per month and product
f = [150000, 150000, 100000]
# Minimum lot size per product and month
l = [20, 20, 16]

In [215]:
# Model definition
model = mip.Model()

In [216]:
# quantity of product i produced in month j
x = {(i, j): model.add_var(var_type=INTEGER, lb=0, name=f"(x_{i},{j})") for i in I for j in J}
# quantity of product i sold in month j
v = {(i, j): model.add_var(var_type=INTEGER, lb=0, name=f"(y_{i},{j})") for i in I for j in J}
# quantity of product i stored at the end of month j
z = {(i, j): model.add_var(var_type=INTEGER, lb=0, name=f"(z_{i},{j})") for i in I for j in range(n_months+1)}

In [217]:
# maximizing the revenue
model.objective = mip.maximize(mip.xsum(v[i, j] * r[i]  for i in I for j in J) -mip.xsum(x[i, j] * c[i] for i in I for j in J) - mip.xsum(z[i, j] * m[i] for i in I for j in J))

In [218]:
# Demand constraint
for i in I:
  for j in J:
    model.add_constr(v[i, j] <= d[i][j-1])


# Production constraint
for j in J:
  model.add_constr(mip.xsum(x[i, j] / q[i] for i in I) <=  b[j-1])

# Balance constraint
for i in I:
  for j in J:
    model.add_constr(x[i, j] - v[i, j] == z[i, j])

# Init constraint
for i in I:
  model.add_constr(z[i, 1] == 0)

# Capacity constraint
for j in J:
  model.add_constr(mip.xsum(z[i, j] for i in I) <= K)

In [219]:
# optimizing
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [220]:
# optimal objective function value
model.objective.x

2338840.0

In [221]:
# print the optimal values of each variables
print("x")
for i in I:
    for j in J:
        print("{:-7}".format(x[i,j].x), end=" ")
    print()
print("v")
for i in I:
    for j in J:
        print(f"{v[i,j].x:-7}", end=" ")
    print()
print("z")
for i in I:
    for j in J:
        print(f"{z[i,j].x:-7}", end=" ")
    print()

x
 2000.0     0.0     0.0     0.0 
 4500.0  3518.0   122.0     0.0 
 4400.0  6700.0 12500.0 12100.0 
v
 2000.0     0.0     0.0     0.0 
 4500.0  3518.0   122.0     0.0 
 4400.0  6700.0 12500.0 12100.0 
z
    0.0     0.0     0.0     0.0 
    0.0     0.0     0.0     0.0 
    0.0     0.0     0.0     0.0 


In [222]:
# definition of the new model
model1 = mip.Model()

In [223]:
# quantity of product i produced in month j
x = {(i, j): model1.add_var(name ='x('+str(i)+','+str(j)+')', var_type=INTEGER, lb=0) for i in I for j in J}
# quantity of product i sold in month j
v = {(i, j): model1.add_var(name ='v('+str(i)+','+str(j)+')', lb=0) for i in I for j in J}
# quantity of product i stored at the end of month j
z = {(i, j): model1.add_var(name ='z('+str(i)+','+str(j)+')', lb=0) for i in I for j in range(n_months+1)}
# binary variable: 1 if product i is produced in month j, 0 else
y = {(i, j): model1.add_var(name ='y('+str(i)+','+str(j)+')', var_type=BINARY) for i in I for j in J}

In [224]:
model1.objective = mip.maximize(mip.xsum(r[i]*v[i, j] - c[i]*x[i, j] - m[i]*z[i, j] - f[i]*y[i, j] for i in I for j in J))

In [225]:
# Demand constraint
for i in I:
  for j in J:
    model1.add_constr(v[i, j] <= d[i][j-1])

# Production constraint
for j in J:
  model1.add_constr(mip.xsum(x[i, j] / q[i] for i in I) <= b[j-1] )

# Balance constraint
for i in I:
  for j in J:
    model1.add_constr(z[i, j-1] + x[i, j] == z[i, j] + v[i, j])

# Init constraint
for i in I:
  model1.add_constr(z[i, 0] == 0)

# Capacity constraint
for j in J:
  model1.add_constr(mip.xsum(z[i, j] for i in I) <= K)

# Activation constraint
for i in I:
  for j in J:
    model1.add_constr(x[i, j] <= b[j-1] * q[i] * y[i, j])

# Lot size constraint
for i in I:
  for j in J:
    model1.add_constr(x[i, j] >= l[i] * y[i, j])

In [226]:
# optimizing
model1.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [227]:
# optimal objective function value
model1.objective.x

1585836.0

In [228]:
# print the optimal values of each variables
print("x")
for i in I:
    for j in J:
        print(f"{x[i,j].x:-7}", end=" ")
    print()
print("v")
for i in I:
    for j in J:
        print(f"{v[i,j].x:-7}", end=" ")
    print()
print("y")
for i in I:
    for j in J:
        print(f"{y[i,j].x:-7.3}", end=" ")
    print()
print("z")
for i in I:
    for j in J:
        print(f"{z[i,j].x:-7}", end=" ")
    print()

x
 6100.0     0.0     0.0     0.0 
    0.0  2988.0     0.0     0.0 
 4400.0  7348.0 12650.0 12100.0 
v
 5300.0   800.0     0.0     0.0 
    0.0  2988.0     0.0     0.0 
 4400.0  6700.0 12500.0 12898.0 
y
    1.0     0.0     0.0     0.0 
    0.0     1.0     0.0     0.0 
    1.0     1.0     1.0     1.0 
z
  800.0     0.0    -0.0     0.0 
    0.0    -0.0     0.0     0.0 
    0.0   648.0   798.0     0.0 
